Make the Binary Quadratic Model for sports scheduling problem.
Definitions and comments in this code are based on the following paper.

Title:
  **SOLVING LARGE BREAK MINIMIZATION PROBLEMS IN A MIRRORED DOUBLE ROUND-ROBIN TOURNAMENT USING QUANTUM ANNEALING.**
  (https://arxiv.org/pdf/2110.07239.pdf)

Author:
 - Michiya Kuramata (Tokyo Institute of Technology, Tokyo, Japan)
 - Ryota Katsuki (NTT DATA, Tokyo, Japan)
 - Nakata Kazuhide (Tokyo Institute of Technology, Tokyo, Japan)

In [1]:
import itertools

import pyqubo
from dwave.system.samplers import DWaveSampler
from dwave.system import EmbeddingComposite
import neal
import gurobipy

from sports.sports_scheduling import *
from sports.sports_gurobi import *

# MDRRT using D-Wave Machine (or SA).

#### Set the Parameters for MDRRT

In [2]:
# 2n is the number of teams.
# n should be 2 <= n <= 24.
# If n=2, the num of teams is 4 and timeslots in RRT and MDRRT are 3 and 6 respectively (same shape with Table 1).
# If n=4, the num of teams is 8 and timeslots in RRT and MDRRT are 7 and 14.
n = 4

# version is the problem instance used in this experiments.
# Each version is correspond to the csv files in problems directory.
# These files are used in Numerical experiments and Discussion section in our paper.
# Csv files are calculated by Kirkman Schedule and shuffled in advance independently.
# If you need to remake them, run sports_scheduling.kirkman_schedule and shuffle them on your own.
# If you set n=2 and choose version=0, you are going to use problems/02_v0_schedule.csv
version = 0

mdrrt = MDRRT(n, f'./problems/{n:02}_v{version}_schedule.csv')

`mdrrt.schedule` is based on MDRRT (**Mirrored** Double Round Robin Tournament), so the table in timeslot 0\~6 and 7\~13 is same.

 - num of teams: 8
 - num of timeslots in MDRRT: 14

In [3]:
mdrrt.schedule

timeslot,0,1,2,3,4,5,6,7,8,9,10,11,12,13
team,,,,,,,,,,,,,,
0,3,7,4,6,5,1,2,3,7,4,6,5,1,2
1,2,6,3,5,4,0,7,2,6,3,5,4,0,7
2,1,5,7,4,3,6,0,1,5,7,4,3,6,0
3,0,4,1,7,2,5,6,0,4,1,7,2,5,6
4,6,3,0,2,1,7,5,6,3,0,2,1,7,5
5,7,2,6,1,0,3,4,7,2,6,1,0,3,4
6,4,1,5,0,7,2,3,4,1,5,0,7,2,3
7,5,0,2,3,6,4,1,5,0,2,3,6,4,1


In [4]:
# Definition of decision variable z
z_q = pyqubo.Array.create('z', shape=n*(2*n-1), vartype='BINARY')

# Definition of decision variable y
y_q = z_to_y(mdrrt, z_q)

In [5]:
# Make the objective function (6) using pyqubo
# Note that you don't need to include constraints here according to (7) and (8).
objective_function = make_objective_function(mdrrt, y_q)
model = objective_function.compile()

# Make the Binary Quadratic Model for D-Wave.
bqm = model.to_bqm()
bqm.normalize()

0.125

#### Input bqm into D-Wave or Simulated Annealing.

In [ ]:
# Set the endpoint, token and solver for the experiments.
# You need to register D-Wave Leap account.
# (If you use Simulated Annealing altenatively, please ignore this block.)
endpoint = 'https://cloud.dwavesys.com/sapi'  # change according to your account.
token = '***'  # change according to your account.
solver = 'Advantage_system1.1'  # change according to your account.

child_sampler = DWaveSampler(
     endpoint=endpoint,
     token=token,
     solver=solver
 )

sampler = EmbeddingComposite(child_sampler)

In [6]:
# If you use Simulated Annealing altenatively, run following block.
# (If you use D-Wave Machine, please ignore this block.)

sampler = neal.SimulatedAnnealingSampler()

In [7]:
# If you use D-Wave Machine, set the appropriate parameters here

num_reads = 1000
annealing_time = 50

sampleset = sampler.sample(
    bqm,
    annealing_time = annealing_time,  #If you use SA, comment out this line.
    num_reads = num_reads
)
sampleset = sampleset.aggregate()

# Pick up top 3 solutions.
data_list = [data for i, data in enumerate(sampleset.data(sorted_by='energy')) if i <3]

break_term_c = objective_function.compile()

samples = break_term_c.decode_sampleset(sampleset)
sample = min(samples, key=lambda s: s.energy)

num_breaks_qa = int(sample.energy)

#### Result

In [8]:
print("num of breaks (D-Wave or SA):", num_breaks_qa)

num of breaks (D-Wave or SA): 20


# MDRRT experiments by Integer Programming (Urdaneta)

In [9]:
model, y = urdaneta_BMP(
    mdrrt,
    timeout=300,
    model_name='Urdaneta_BMP'
)

num_breaks_urdaneta = round(model.objval)
calculation_time = model.Runtime

Restricted license - for non-production use only - expires 2022-01-13
No parameters matching 'timeout' found
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 0 rows, 28 columns and 0 nonzeros
Model fingerprint: 0x37ab5cd4
Model has 56 quadratic objective terms
Variable types: 0 continuous, 28 integer (28 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [2e+00, 8e+00]
  QObjective range [4e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 66.0000000
Presolve time: 0.00s
Presolved: 56 rows, 84 columns, 168 nonzeros
Variable types: 0 continuous, 84 integer (84 binary)

Root relaxation: objective 0.000000e+00, 27 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time



In [10]:
print("num of broken (Urdaneta):", num_breaks_urdaneta)
print(f'time (Urdaneta): {round(calculation_time, 7)} seconds')

num of broken (Urdaneta): 20
time (Urdaneta): 0.319375 seconds


# MDRRT experiments by Integer Programming (Trick)

In [11]:
model, athome = trick_BMP(
    mdrrt,
    timeout=300,
    model_name='Trick_BMP'
)

calculation_time = model.Runtime

gb_answer = np.zeros((mdrrt.num_teams, mdrrt.num_slots), int)
for i in range(mdrrt.num_teams):
    for t in range(mdrrt.num_slots):
        try:
            temp = athome[i][t].getValue() if isinstance(athome[i][t], gurobipy.LinExpr) else athome[i][t].X
            gb_answer[i][t] = round(temp, 2)
        except AttributeError:
            break

num_breaks_trick = 0
for i in range(mdrrt.num_teams):
    prev = None
    for t in gb_answer[i]:
        if prev == t:
            num_breaks_trick += 1
        prev = t

Changed value of parameter MIPGapAbs to 1.99
   Prev: 1e-10  Min: 0.0  Max: inf  Default: 1e-10
No parameters matching 'timeout' found
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 717 rows, 216 columns and 10385 nonzeros
Model fingerprint: 0x0abf4740
Variable types: 0 continuous, 216 integer (216 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 196 rows and 11 columns
Presolve time: 0.09s
Presolved: 521 rows, 205 columns, 6528 nonzeros
Variable types: 0 continuous, 205 integer (205 binary)
Found heuristic solution: objective 56.0000000

Root relaxation: objective 8.720000e+01, 320 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/No

In [12]:
print(f'breaks (Trick): {num_breaks_trick}')
print(f'time (Trick): {round(calculation_time, 7)} seconds')

breaks (Trick): 20
time (Trick): 0.7133479 seconds


# Compare methods

The calculation time for Gurobi to reach the objective function value which the D-Wave Advantage reaches in 0.05 s.

In [13]:
#Terminate if IP(urdaneta) find the better solution than QA.
#solve the problem using gurobi(IP(urdaneta))
model,y = urdaneta_BMP(
    mdrrt,
    timeout=None,
    model_name='Urdaneta_BMP',
    best_obj_stop=num_breaks_qa
)

urdaneta_time = model.Runtime

No parameters matching 'timeout' found
Changed value of parameter best_obj_stop to 20.0
   Prev: -inf  Min: -inf  Max: inf  Default: -inf
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 0 rows, 28 columns and 0 nonzeros
Model fingerprint: 0x37ab5cd4
Model has 56 quadratic objective terms
Variable types: 0 continuous, 28 integer (28 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [2e+00, 8e+00]
  QObjective range [4e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 66.0000000
Presolve time: 0.00s
Presolved: 56 rows, 84 columns, 168 nonzeros
Variable types: 0 continuous, 84 integer (84 binary)

Root relaxation: objective 0.000000e+00, 27 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    

In [14]:
print(f"time: {round(urdaneta_time,7)} seconds")

time: 0.1600399 seconds
